# Création du dataset d'alphabet de langue des signes

La première étape du document consiste à créer un dataset un prenant des photos de mains faisant les signes demandés. Le notebook sera donc à relancer pour la création de chaque lettre.

## Capture des images de la lettre

Importer les modules nécessaires :

In [1]:
import cv2
import numpy as np
from pathlib import Path

Choisir la lettre à faire signer :

In [2]:
element = input()

A


Créer le dossier de réception s'il n'existe pas :

In [3]:
Path("gesture\\" + str(element)).mkdir(parents=True, exist_ok=True)

Définition de variables (variable booléenne``background``, ``accumulated_weight``, et les limites du cadre de prise d'image) :

In [4]:
background = None
accumulated_weight = 0.5

ROI_top = 100
ROI_bottom = 300
ROI_right = 150
ROI_left = 350

Définition d'une fonction de calcul à partir de la fonction ``accumulated_weight`` de ``cv2``, pour collecter en image la moyenne de l'output de la caméra.

In [5]:
def cal_accum_avg(frame, accumulated_weight):

    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None

    cv2.accumulateWeighted(frame, background, accumulated_weight)

Cette image définira le background, qui servira à différencier la main du reste de l'image en calculant les différences avec la première.

Définition d'une fonction de prise en image des contours de la main, avec ``absdiff``, ``threshold`` et `` findCoutours`` de ``cv2`` :

In [6]:
def segment_hand(frame, threshold=25):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)

    _ , thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)

    # Grab the external contours for the image
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return None
    else:
        
        hand_segment_max_cont = max(contours, key=cv2.contourArea)
        
        return (thresholded, hand_segment_max_cont)

Démarrage de la webcam et capture des images :

In [7]:
cam = cv2.VideoCapture(0)

num_frames = 0
num_imgs_taken = 0

while True:
    ret, frame = cam.read()

    #Faire pivoter les images
    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()

    #Définition du cadre de capture avec son carré à afficher
    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]
    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)

    #Capture du background
    if num_frames < 60:
        cal_accum_avg(gray_frame, accumulated_weight)
        if num_frames <= 59:
            
            cv2.putText(frame_copy, "Capture du background, en attente...", (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
         
    #Détection de la main dans le cadre
    ##Si le nombre d'images prises dépasse 100...
    elif num_frames <= 100: 

        hand = segment_hand(gray_frame)
        
        cv2.putText(frame_copy, "Ajuster la main pour le signe de " + str(element), (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        
        #Vérification de la présence de la main en détectant les contours
        if hand is not None:
            
            thresholded, hand_segment = hand

            #Dessiner les contours autour de la main
            cv2.drawContours(frame_copy, [hand_segment + (ROI_right, ROI_top)], -1, (255, 0, 0),1)
            
            cv2.putText(frame_copy, str(num_frames)+" pour " + str(element), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

            #Montrer l'image de la main version threshold
            cv2.imshow("Threshold de main", thresholded)
    
    ##Tant que le nombre d'images prises ne dépasse pas 100...
    else: 
        
        # Segmenter la région de la main
        hand = segment_hand(gray_frame)
        
        #Vérification de la présence de la main
        if hand is not None:
            
            # Obtenir le thresholded et le max_contour de la main
            thresholded, hand_segment = hand

            #Dessiner les contours autour de la main
            cv2.drawContours(frame_copy, [hand_segment + (ROI_right, ROI_top)], -1, (255, 0, 0),1)
            
            #Ajouter le texte
            cv2.putText(frame_copy, str(num_frames), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            #cv2.putText(frame_copy, str(num_frames)+"pour" + str(element), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.putText(frame_copy, str(num_imgs_taken) + 'images' +"pour" + str(element), (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
            #Montrer l'image de la main version threshold
            cv2.imshow("Threshold de main", thresholded)
            if num_imgs_taken <= 100:
                #Enregistrer l'image
                cv2.imwrite("gesture\\" + str(element) + "\\" + str(element)  + str(num_imgs_taken) + '.jpg', thresholded)
            else:
                break
            num_imgs_taken +=1
        #Si la main n'est pas détectée...
        else:
            cv2.putText(frame_copy, 'Pas de main en vue...', (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    #Dessiner le cadre sur la copie de l'image
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right, ROI_bottom), (255,128,0), 3)
    
    #Ajouter le texte de description
    cv2.putText(frame_copy, "Reconaissance de langue des signes DataFlair", (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    
    #Augmenter le nombre de frames pour le compte
    num_frames += 1

    #Montrer l'image avec la main segmentée
    cv2.imshow("Detection de signe", frame_copy)

    #Utiliser ESCAPE pour arrêter
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

#Fermer la caméra et les fenêtres
cv2.destroyAllWindows()
cam.release()

## Correction manuelle

Les images créées, on peut aller les vérifier dans le dossiers et retirer celles qui paraissent incorrectes. Le nombre d'image peut être réduit sans trop de problème car il leur sera appliqué une data augmentation dans la partie suivante. Penser à garder un nombre similaire d'images par lettre pour l'équilibre du dataset.